<a href="https://colab.research.google.com/github/mlmaniac-neelothkulaambaal/MLProjects/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Download text from given URL**

In [39]:
!wget https://ditotw.space/NASA_access_log_Jul95.gz

--2022-02-07 12:44:26--  https://ditotw.space/NASA_access_log_Jul95.gz
Resolving ditotw.space (ditotw.space)... 162.241.217.135
Connecting to ditotw.space (ditotw.space)|162.241.217.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20676672 (20M) [application/x-gzip]
Saving to: ‘NASA_access_log_Jul95.gz’

NASA_access_log_Jul 100%[===================>]  19.72M  7.00MB/s    in 2.8s    

2022-02-07 12:44:29 (7.00 MB/s) - ‘NASA_access_log_Jul95.gz’ saved [20676672/20676672]



**2. Install pyspark and importing necessary libraries**

In [40]:
pip install pyspark

In [41]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
import os.path
import re

**3. Confirm that the download is successful. The file is in the local working direrctory in Google Colab.**

In [75]:
mypath='/content/NASA_access_log_Jul95.gz'
os.path.isfile(mypath)

True

**4. Create sparkcontext and RDD**

In [76]:
config = SparkConf().setAppName("NASA_Logs").setMaster("local[*]")
sc = SparkContext.getOrCreate(config)
sqlcontext = SQLContext(sc)
rdd = sc.textFile(mypath)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


**5. Check number of rows in the RDD**

In [77]:
rdd.count()

1891715

**6. Printing first 5 records in the RDD**

In [78]:
rdd.take(1)

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245']

In [79]:
i=0
for line in rdd.sample(withReplacement = False, fraction = 0.00001, seed = 100).collect():
    i=i+1
    print(line)
    if i >5:
      break

ix-lb7-05.ix.netcom.com - - [02/Jul/1995:04:04:00 -0400] "GET /shuttle/missions/sts-71/images/images.html HTTP/1.0" 200 7634
zachar.fast.net - - [03/Jul/1995:00:06:07 -0400] "GET /shuttle/technology/images/sts_body_2-small.gif HTTP/1.0" 200 30067
ix-bos8-08.ix.netcom.com - - [03/Jul/1995:11:48:45 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 200 234
199.120.22.3 - - [03/Jul/1995:21:24:27 -0400] "GET /shuttle/missions/sts-67/sts-67-patch-small.gif HTTP/1.0" 200 17083
wstabnow.clark.net - - [04/Jul/1995:13:37:33 -0400] "GET /shuttle/technology/images/srb_mod_compare_1-small.gif HTTP/1.0" 200 36902
alyssa.prodigy.com - - [05/Jul/1995:23:59:57 -0400] "GET /history/apollo/apollo-4/images/ HTTP/1.0" 200 514


In [80]:
def parse_log1(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([\w\.\s*]+)?\s*"*(\d{3}) (\S+)', line)
    if match is None:
        return 0
    else:
        return 1

n_logs = rdd.count()

failed = rdd.map(lambda line: parse_log1(line)).filter(lambda line: line == 0).count()

print('Out of {} logs, {} failed to parse'.format(n_logs,failed))

Out of 1891715 logs, 855 failed to parse


In [81]:
def parse_log2(line):

    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)

failed = rdd.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 0).count()

print('Out of {} logs, {} failed to parse'.format(n_logs,failed))

Out of 1891715 logs, 622 failed to parse


In [82]:
def map_log(line):

    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    
    return(match.groups())

In [83]:
parsed_rdd = rdd.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])

In [84]:
parsed_rdd2 = parsed_rdd.map(lambda line: map_log(line))

In [85]:
for i in parsed_rdd2.take(3):
    print(i)
    print('\n')

('199.72.81.55', '-', '-', '01/Jul/1995:00:00:01', '0400', 'GET', '/history/apollo/', 'HTTP/1.0"', None, '200', '6245')


('unicomp6.unicomp.net', '-', '-', '01/Jul/1995:00:00:06', '0400', 'GET', '/shuttle/countdown/', 'HTTP/1.0"', None, '200', '3985')


('199.120.110.21', '-', '-', '01/Jul/1995:00:00:09', '0400', 'GET', '/shuttle/missions/sts-73/mission-sts-73.html', 'HTTP/1.0"', None, '200', '4085')




In [86]:
parsed_rdd2.map(lambda line: len(line)).distinct().collect()

[11]

In [105]:
def result(n):
  result = parsed_rdd2.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b).takeOrdered(n, lambda x: -x[1])
  print(result)

In [106]:
n=input('Enter top N sites you wish to see:')

result(int(n))


Enter top N sites you wish to see:3
[('piweba3y.prodigy.com', 17572), ('piweba4y.prodigy.com', 11591), ('piweba1y.prodigy.com', 9868)]
